<br><H3>Feature: Building Web Scraper to get related news blogs, articles related to User-input keywords</H3></br>
<br></br>
Author: Ronak Dedhia
<br></br>
Version: 1.1
<br></br>
Date: 22 September, 2018
<br></br>
<H5>Revision 1:</H5>
Revision Date: 23 September, 2018
<br></br>
Take multiple keywords from user and display only top 5 articles of each keyword
<br></br>
<H5>Revision 2:</H5>
Revision Date: 28 September, 2018
<br></br>
Introduce exception handling to limit the number of keywords and notifications if no articles found, invalid number


In [40]:
#Installing the required packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install bs4


In [27]:

#Importing required Packages
import pandas as pd
from collections import OrderedDict
from datetime import datetime
import urllib
import requests
from bs4 import BeautifulSoup

#Set all lists to capture different elements of the search
kywd = []
sr_no = []
news_title = []
news_dt = []
news_link = []
news_kywd = []
raw_keyword = []  # For Displaying the topic

# Get user input of keywords for which they want to recieve alerts
while True:
    try:
        kywd_cnt = int(input('How many key words do you want(Maximum 5 keywords)? '))
        if(kywd_cnt < 6 and kywd_cnt != 0):
            break
        print('Enter atleast 1 or upto 5 keywords')
    except ValueError:
        print("No valid integer! Please try again ...")

for i in range(kywd_cnt):
    # 1.Take input  2. Quote it to fit the url format 3. Append it to keywd variable
    raw_keyword.append(input('Enter keyword {}:'.format(i+1)))
    kywd.append(urllib.parse.quote(raw_keyword[i]))
    
for x in range(len(kywd)):
    #Request seperate web page for results based on different keywords    
    page = requests.get("http://www.denofgeek.com/search/site/{}".format(kywd[x]))
    page_status_code = page.status_code                   #Value 200 indicates page downloaded successfully
    soup = BeautifulSoup(page.content, 'html.parser')     #Create instance of Beautiful Soup
    #Filter all the tags containing the search articles
    outer_list = soup.find_all('li', class_= 'search-result clearfix')
    
    #Check and notify if no articles exist with the keywords
    if not outer_list:
        print('No results found for the keyword "{}"'.format(raw_keyword[x]))
        
    #Loop through the tags inside lists to get required data
    for article in enumerate(outer_list):
        dt = article[1].find('div', class_='date')
        link = article[1].div.find('a', href=True)
        if link is not None:
            sr_no.append(article[0] + 1)
            news_title.append(article[1].h2.text.strip())
            news_link.append(link['href'].strip())
            news_dt.append(dt.text.strip())
            news_kywd.append(raw_keyword[x])

#Defining the dataframe for storing topics
news_table = {
            #'sr_no':sr_no,
            'news_title':news_title,
            'news_date':news_dt,
             'news_link': news_link,
            'news_keyword': news_kywd}
news_table['news_date_format'] = pd.to_datetime(news_table['news_date'], format='%b %d, %Y', errors='ignore')
news_table = pd.DataFrame(OrderedDict(news_table))

#Sort the values based on datetime and topic
news_table.sort_values(by=['news_keyword', 'news_date_format'],ascending= False, inplace= True)

#Drop the datetime column after sorting is done
news_table.drop('news_date_format', axis=1, inplace= True)

#Display the pretty version of the extracted data (Top 5 or less depending on the topic)
pd.set_option('display.max_colwidth', -1) #Show entire text for article links

news_table.groupby('news_keyword').head(5).reset_index(drop=True)

How many key words do you want(Maximum 5 keywords)? 
No valid integer! Please try again ...
How many key words do you want(Maximum 5 keywords)? 7
Enter atleast 1 or upto 5 keywords
How many key words do you want(Maximum 5 keywords)? 8
Enter atleast 1 or upto 5 keywords
How many key words do you want(Maximum 5 keywords)? @
No valid integer! Please try again ...
How many key words do you want(Maximum 5 keywords)? 0
Enter atleast 1 or upto 5 keywords
How many key words do you want(Maximum 5 keywords)? 1
Enter keyword 1:we


,news_date,news_title,news_keyword,news_link
0,"Sep 24, 2018",EGX 2018: Our standout games from the show,we,http://www.denofgeek.com/uk/games/egx-2018/60651/egx-2018-our-standout-games-from-the-show
1,"Sep 21, 2018","Eli Roth interview: ""we had the ‘Potter Police’ on set""",we,http://www.denofgeek.com/uk/movies/eli-roth/60574/eli-roth-interview-we-had-the-potter-police-on-set
2,"Sep 21, 2018",Netflix’s new show Maniac asks “how do we feel better?”,we,http://www.denofgeek.com/uk/tv/maniac/60596/netflix-s-new-show-maniac-asks-how-do-we-feel-better
3,"Sep 13, 2018",Star Trek: the future of the movies and what we know,we,http://www.denofgeek.com/uk/movies/star-trek/60418/star-trek-the-future-of-the-movies-and-what-we-know
4,"Jul 19, 2018",Mamma Mia! Here We Go Again review,we,http://www.denofgeek.com/uk/movies/mamma-mia-here-we-go-again/59202/mamma-mia-here-we-go-again-review
